In [ ]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics

In [ ]:
# To print full list
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
%%time

import pymssql
import os
from dotenv import load_dotenv

load_dotenv()

server = os.getenv('SERVER')
user = os.getenv('USER')
password = os.getenv('PASSWORD')

conn = pymssql.connect(server, user, password, "chillipadiad_db")
cursor = conn.cursor(as_dict=True)

cursor.execute('SELECT * FROM [Order]')
orderData = pd.DataFrame(cursor)
orderData.columns=[ x[0] for x in cursor.description]

cursor.execute('SELECT * FROM Customer')
customerData = pd.DataFrame(cursor)
customerData.columns=[ x[0] for x in cursor.description]

conn.close()

In [ ]:
orderData.head()

# Removing cols with major null vals

In [ ]:
def naValsInPercentage(data):
    percent_missing = data.isnull().sum() * 100 / len(data)
    return pd.DataFrame({'column_name': data.columns,
                         'percent_missing': percent_missing})
missingVals = naValsInPercentage(orderData)
missingVals

In [ ]:
# remove columns with more than 30% missing values
colsToDrop = missingVals[missingVals['percent_missing']>30]
colsToDrop

### Some columns are still important to analyse
DateOfBirth, AddOnsPrice

In [ ]:
finalColsToDrop = colsToDrop.drop(['AddOnsPrice'])

In [ ]:
# CD = Cleaned Data
cdPart1 = orderData.drop(columns = finalColsToDrop['column_name'])

## Drop Rows without main identifier
# We are using Menu names as the main identifier
rowsToDrop = cdPart1[cdPart1["MenuName"].isna()]
cdPart1 = cdPart1.drop(index = rowsToDrop.index)

cdPart1.info()

### Dropping more insignificant columns

In [ ]:
# Postal code sufficient for location, hence drop other location related data
moreColsToDrop = ['Block', 'Street', 'Building']
# invoice id?
moreColsToDrop.insert(0,'InvoiceId')
# with driver id, driver name unnecessary
moreColsToDrop.insert(0,'DriverName')
# OrderSource all HQ so quite useless
moreColsToDrop.insert(0,'OrderSource')

# finally remove those cols
cdPart2 = cdPart1.drop(columns = moreColsToDrop)
len(cdPart2.columns)

# Add in customer DOB

In [ ]:
customerData.info()

In [ ]:
colsToKeep = ['CustomerId', 'Company', 'DateOfBirth']
strippedCustomerData = pd.DataFrame(customerData[colsToKeep])
strippedCustomerData.head()

In [ ]:
mergedData = pd.merge(cdPart2, strippedCustomerData, on='CustomerId', how='left')

In [ ]:
mergedData.info()

### Fill Null values

In [ ]:
missingVal = naValsInPercentage(mergedData)
missingVal

In [ ]:
# DateOfBirth

from datetime import datetime, date
cdPart3 = mergedData.copy()

cdPart3['DateOfBirth'] = pd.to_datetime(cdPart3['DateOfBirth'], format='%d/%m/%Y %H:%M')

cdPart3['DateOfBirth(Date)'] = cdPart3['DateOfBirth'].dt.strftime('%d/%m/%Y')
cdPart3['DateOfBirth(Date)'] = cdPart3['DateOfBirth'].replace(np.nan, date.today().strftime("%d/%m/%Y")).astype(str)

cdPart3= cdPart3.drop(columns=['DateOfBirth'])

In [ ]:
# fill the rest

cdPart3[['AddOnsPrice','DeliveryRateInternal']] = cdPart3[['AddOnsPrice','DeliveryRateInternal']].fillna(value=0)

negOne = ['PackedTime','StaffPrice','StaffNo','DeliveryRate','DriverId']
cdPart3[negOne] = cdPart3[negOne].fillna(value=-1)

none = ['Company','DeliveryNote','ContactPerson','EventType','FunctionDate']
cdPart3[none] = cdPart3[none].fillna(value='none')

cdPart3[['PostalCode','ContactNumber']] = cdPart3[['PostalCode','ContactNumber']].fillna(value='0')



In [ ]:
cdPart3.sample(10)

In [ ]:
missingVal = naValsInPercentage(cdPart3)
missingVal

In [ ]:
cdPart3.to_csv('data/chillipadi_cleandata.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')